Uploaded for the sake of uploading

In [90]:
import pandas as pd
import pyarrow
from dotenv import load_dotenv
import os

load_dotenv()

file_path = os.getenv('FILE_PATH')

filenames = [
    'bag_ids_no_funda',
    'random_online_sample',
    'special_house_types_class_sample',
    'detailed_woning_type_sample',
    'not_online_listings', # Download from slack
]

In [91]:
print(file_path)

../../Samsung_USB/


In [92]:
for filename in filenames:
    filename = filename + '.parquet' if not filename.endswith('.parquet') else filename	
    df = pd.read_parquet(file_path + filename, engine='pyarrow')
    df.to_csv(
        file_path + filename.replace('.parquet', '.csv'),
        index=False,
        encoding='utf-8',
    )

In [93]:
print(filenames)

# When printing the content of the CSV files, the file size becomes too large to be pushed to GitHub.

# for filename in filenames:
#     filename = filename + '.csv' if not filename.endswith('.csv') else filename
#     print(f"Reading file: {filename}")
#     with open(file_path + filename, 'r', encoding='utf-8') as file:
#         content = file.read()
#         print(content)

['bag_ids_no_funda', 'random_online_sample', 'special_house_types_class_sample', 'detailed_woning_type_sample', 'not_online_listings']


In [94]:
import pandas as pd

df = pd.read_csv(file_path + "bag_image_summary.csv")
df_sample = pd.read_parquet(file_path + "detailed_woning_type_sample.parquet")
df = pd.read_csv(file_path + "bag_image_summary.csv", dtype="string")
df_joined = pd.merge(df_sample, df, how="left", right_on="bag_id", left_on="bag_nummeraanduidingid")
df_sample_with_urls = df_joined[df_joined["frontview_exists"].notna()]

file_path_with_urls = False


In [95]:
import re

def extract_path(url, source):
    if pd.isna(url):
        return ''
    id = url.rstrip('/').split('/')[-1]
    id, _ = id.split('.')
    m = re.match(r'(\d{2})', id)
    first_two_digits = m.group(1) if m else ''
    return f"img_dataset/{first_two_digits}/{id}-{source}.jpg"

cols = ['frontview_funda_url', 'frontview_google_url', 'frontview_funda_in_business_url']
sources = ['funda', 'google', 'funda-in-business']

for col, source in zip(cols, sources):
    df_sample_with_urls[f'{col}_split'] = df_sample_with_urls[col].map(lambda url: extract_path(url, source))


In [96]:
if file_path_with_urls == False:
    df_sample_with_urls[[f'{col}_split' for col in cols]] = df_sample_with_urls[[f'{col}_split' for col in cols]].map(lambda x: file_path + x if x else '')
    file_path_with_urls = True

df_sample_with_urls.to_csv(
    file_path + "detailed_woning_type_with_urls.csv",
    index=False,
    encoding='utf-8',
)
